# HuggingFace project

## Text-to-Text (summarization)

In [ ]:
!pip install transformers datasets
!pip install accelerate -U
!pip install certifi -U
!pip install evaluate -U
!pip install rouge_score -U
!pip install wandb -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=0f9a2230a0f5bca252f44d1fa0a122b66dfae253cf5e2e85a9e587752bd5d24a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
import transformers
from transformers import pipeline

In [ ]:
# # load model (model "facebook/bart-base") to pipeline
# pipe = pipeline("summarization", model="facebook/bart-base")

# # sample model
# pipe(
#     """
#     Education is a fundamental pillar of society that empowers individuals with the knowledge, skills, and values needed for personal and professional development.
#     It serves as a pathway to economic opportunities, fosters critical thinking, and promotes social equality.
#     Formal education typically includes primary, secondary, and higher education, while informal learning happens throughout life.
#     In today's digital age, technology is transforming the educational landscape, offering innovative methods for teaching and learning, such as online courses and interactive tools.
#     However, challenges like access to quality education and educational inequality still persist, necessitating ongoing efforts to ensure inclusive and equitable learning for all.
#     """
# )

In [ ]:
# pipe = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum")

# pipe(
#     """
#     Education is a fundamental pillar of society that empowers individuals with the knowledge, skills, and values needed for personal and professional development.
#     It serves as a pathway to economic opportunities, fosters critical thinking, and promotes social equality.
#     Formal education typically includes primary, secondary, and higher education, while informal learning happens throughout life.
#     In today's digital age, technology is transforming the educational landscape, offering innovative methods for teaching and learning, such as online courses and interactive tools.
#     However, challenges like access to quality education and educational inequality still persist, necessitating ongoing efforts to ensure inclusive and equitable learning for all.
#     """
# )

## Load dataset for training model

In [ ]:
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import numpy as np

In [ ]:
# load_dataset (Multi-News dataset)
raw_dataset = load_dataset("multi_news", trust_remote_code=True)

# checkpoint = "sshleifer/distilbart-cnn-6-6"
checkpoint = "facebook/bart-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

multi_news.py:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

train.src.cleaned:   0%|          | 0.00/548M [00:00<?, ?B/s]

train.tgt:   0%|          | 0.00/58.8M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/460M [00:00<?, ?B/s]

In [ ]:
if torch.cuda.is_available():
  print(torch.version.cuda)
  model.to('cuda')

12.1


In [ ]:
# data in dataset (train, validation, test)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 44972
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 5622
    })
})

In [ ]:
# sample data
raw_dataset['train'][0]

{'document': 'National Archives \n \n Yes, it’s that time again, folks. It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. \n \n A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m. New York time offering one of the most important snapshots on how the economy fared during the previous month. Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February. The unemployment rate is expected to hold steady at 8.3%. \n \n Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires. Feel free to weigh-in yourself, via the comments section. And while you’re here, why don’t you sign up to follow us on Twitter. \n \n Enjoy the show. ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy 

In [ ]:
raw_dataset['train'][1]

{'document': 'LOS ANGELES (AP) — In her first interview since the NBA banned her estranged husband, Shelly Sterling says she will fight to keep her share of the Los Angeles Clippers and plans one day to divorce Donald Sterling. \n \n (Click Prev or Next to continue viewing images.) \n \n ADVERTISEMENT (Click Prev or Next to continue viewing images.) \n \n Los Angeles Clippers co-owner Shelly Sterling, below, watches the Clippers play the Oklahoma City Thunder along with her attorney, Pierce O\'Donnell, in the first half of Game 3 of the Western Conference... (Associated Press) \n \n Shelly Sterling spoke to Barbara Walters, and ABC News posted a short story with excerpts from the conversation Sunday. \n \n NBA Commissioner Adam Silver has banned Donald Sterling for making racist comments and urged owners to force Sterling to sell the team. Silver added that no decisions had been made about the rest of Sterling\'s family. \n \n According to ABC\'s story, Shelly Sterling told Walters: "I

In [ ]:
def preprocess_data(examples):
    document_text = examples['document']
    summary_text = examples['summary']
    return {
        'document_text': document_text,
        'summary_text': summary_text
    }

In [ ]:
train_data_subset = raw_dataset['train'].shuffle(seed=42).select(range(4000))
validation_data_subset = raw_dataset['validation'].shuffle(seed=42).select(range(1000))

# train_data_subset = raw_dataset['train']
# validation_data_subset = raw_dataset['validation']

In [ ]:
train_data = train_data_subset.map(preprocess_data, batched=True, remove_columns=['document', 'summary'])
validation_data = validation_data_subset.map(preprocess_data, batched=True, remove_columns=['document', 'summary'])

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
def tokenize_data(examples):
    inputs = tokenizer(examples['document_text'], max_length=1024, truncation=True, padding=True)
    targets = tokenizer(examples['summary_text'], max_length=128, truncation=True, padding=True)
    inputs['labels'] = targets['input_ids']
    return inputs

In [ ]:
token_train_data = train_data.map(tokenize_data, batched=True)
token_validation_data = validation_data.map(tokenize_data, batched=True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
import wandb
wandb.login(key="18bd1a995e7210e7aa30e5b381892baff012beb7", relogin=True)
wandb.init()

# ตั้งค่าการฝึก
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    run_name="bart-multi-news",
    fp16=True,
)

# สร้าง Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=token_train_data,
    eval_dataset=token_validation_data,
    tokenizer=tokenizer,
)

# เริ่มฝึกโมเดล
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peerasanti-s (peerasanti-s-khon-kaen-university). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.682200,2.527141
2,2.553200,2.502649
3,2.060700,2.533059


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=6000, training_loss=2.3362757835388184, metrics={'train_runtime': 1696.5654, 'train_samples_per_second': 7.073, 'train_steps_per_second': 3.537, 'total_flos': 1.3002778607616e+16, 'train_loss': 2.3362757835388184, 'epoch': 3.0})

In [ ]:
model.save_pretrained("facebook/bart-base_result")
tokenizer.save_pretrained("tokenizer")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.json',
 'tokenizer/merges.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
test_data_subset = raw_dataset['test'].shuffle(seed=42).select(range(200))

In [ ]:
test_data = test_data_subset.map(preprocess_data, batched=True, remove_columns=['document', 'summary'])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
token_test_data = test_data.map(tokenize_data, batched=True)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
result = trainer.evaluate(token_test_data)

In [ ]:
result

{'eval_loss': 2.528510093688965,
 'eval_runtime': 4.8231,
 'eval_samples_per_second': 41.467,
 'eval_steps_per_second': 20.733,
 'epoch': 3.0}

In [ ]:
from evaluate import load
metric = load("rouge")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
predictions = trainer.predict(token_test_data)

In [ ]:
decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in np.argmax(predictions.predictions[0], axis=-1)]
decoded_refs = [tokenizer.decode(ref, skip_special_tokens=True) for ref in predictions.label_ids]
results = metric.compute(predictions=decoded_preds, references=decoded_refs, use_stemmer=True)

In [ ]:
results

{'rouge1': 0.563586877267116,
 'rouge2': 0.2445467618271729,
 'rougeL': 0.4632602100192702,
 'rougeLsum': 0.46300147635144817}

In [ ]:
# text_to_summarize="""
# Climate change refers to significant changes in global temperatures and weather patterns over time. While climate change is a natural phenomenon, human activities, particularly the burning of fossil fuels and deforestation, have accelerated these changes. The consequences are profound, affecting ecosystems and species around the world.
# One major impact of climate change is the loss of biodiversity. As temperatures rise, many species struggle to adapt to the changing environment. For example, polar bears are losing their habitat due to melting ice in the Arctic, leading to a decline in their population. Similarly, coral reefs, which support a diverse range of marine life, are suffering from bleaching caused by higher ocean temperatures.
# Moreover, climate change disrupts the balance of ecosystems. Changes in temperature and precipitation patterns can lead to shifts in species distribution. Some plants may bloom earlier in the spring, while others may struggle to survive. This can create mismatches in food availability for animals, disrupting food webs and leading to population declines.
# In conclusion, climate change poses a significant threat to global ecosystems. It is essential to take immediate action to reduce greenhouse gas emissions and protect our planet's biodiversity. Only through concerted global efforts can we mitigate the impacts of climate change and ensure a sustainable future for all living organisms.
# """


In [ ]:
text_to_summarize = input("Enter text to summarize: ")

Enter text to summarize: One day in the 1920’s, a young man asked if he could visit a woman at her home. When he arrived at her home, she was wearing a hat on her head.   Many people today may not understand what this story means. But any American in the 1920’s would understand. The hat said that the girl would like to go out with him.   In the past, men liked sitting in the girl’s house to talk, to meet her mother, to have something to eat and to listen to her play the piano. However women would like to go on a date, to go out somewhere interesting.   This new style of dating began to take the place of the old system of visiting in society in the early 20th century. Nowadays, it is very popular around the world.


In [ ]:
inputs = tokenizer(text_to_summarize, return_tensors="pt", max_length=1024, truncation=True)
inputs = {key: value.to('cuda') for key, value in inputs.items()}
summary_ids = trainer.model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
print("Original text:", text_to_summarize)
print("Summary:", summary)

Original text: One day in the 1920’s, a young man asked if he could visit a woman at her home. When he arrived at her home, she was wearing a hat on her head.   Many people today may not understand what this story means. But any American in the 1920’s would understand. The hat said that the girl would like to go out with him.   In the past, men liked sitting in the girl’s house to talk, to meet her mother, to have something to eat and to listen to her play the piano. However women would like to go on a date, to go out somewhere interesting.   This new style of dating began to take the place of the old system of visiting in society in the early 20th century. Nowadays, it is very popular around the world.
Summary: – In the 1920s, a young man asked if he could visit a woman at her home. When he arrived at her house, she was wearing a hat on her head. The hat said that the girl would like to go out with him. In the past, men liked sitting in the girl's house to talk, to meet her mother, to